In [1]:
import pandas as pd
import numpy as np

In [6]:
def convert_to_u_v(speed,direction): # Suggested by Github Copilot
    # TODO: check this part
    u = -speed * np.sin(np.deg2rad(direction))
    v = -speed * np.cos(np.deg2rad(direction))
    return u,v

def calculate_depth_averaged_velocities(data): # Frtp's suggestion: Is it just an unweighted mean?
    data['u'],data['v'] = convert_to_u_v(data['Current speed'],data['Current direction'])
    for idx in data.index.unique():
        # Loop over indices. We can't use a rolling mean with a window, because the
        # window differs
        data.loc[idx,'u_depth_averaged'] = data.loc[idx,'u'].mean()
        data.loc[idx,'v_depth_averaged'] = data.loc[idx,'v'].mean()
    # Round data
    data = data.round({'u_depth_averaged': 2, 'v_depth_averaged': 2})
    return data

In [7]:
swedish_to_english = { 'Strömriktning':"Current direction", 'Kvalitet':"Quality_d",'Mätdjup (m)':"Measuring depth (m)_d",
                            'Strömhastighet':"Current speed", 'Kvalitet.1':"Quality_s",'Mätdjup (m).1':"Measuring depth (m)_s"}

## Flinten7 station

In [8]:
Flinten7 = pd.read_csv("raw_data/smhi-opendata_2_3_35137_20241118_065734.csv",index_col=0,sep=";", parse_dates=True)
Flinten7.rename(columns = swedish_to_english,inplace=True)

# Filter data after time steps and quality flags.
Flinten7 = Flinten7.loc[(Flinten7.index >= "2014-01-01") & (Flinten7["Quality_d"] == "G") & (Flinten7["Quality_s"] == "G") ].drop(columns=["Quality_d","Quality_s","Unnamed: 7","Tidsutsnitt"])
Flinten7


,Current direction,Measuring depth (m)_d,Current speed,Measuring depth (m)_s
Datum Tid (UTC),,,,
2014-01-01 00:00:00,4.27,0,57.52,0.0
2014-01-01 00:00:00,32.56,2,47.79,2.0
2014-01-01 00:00:00,33.09,4,47.33,4.0
2014-01-01 00:00:00,36.80,6,49.19,6.0
2014-01-01 01:00:00,8.65,0,61.63,0.0
...,...,...,...,...
2024-11-17 22:00:00,224.35,6,32.92,6.0
2024-11-17 23:00:00,150.49,0,7.72,0.0
2024-11-17 23:00:00,197.60,2,35.49,2.0


In [9]:
# Convert Current speed and directions to u and v components and calculate depth averaged velocities
Flinten7 = calculate_depth_averaged_velocities(Flinten7)
Flinten7


,Current direction,Measuring depth (m)_d,Current speed,Measuring depth (m)_s,u,v,u_depth_averaged,v_depth_averaged
Datum Tid (UTC),,,,,,,,
2014-01-01 00:00:00,4.27,0,57.52,0.0,-4.282743,-57.360339,-21.33,-44.17
2014-01-01 00:00:00,32.56,2,47.79,2.0,-25.719742,-40.278766,-21.33,-44.17
2014-01-01 00:00:00,33.09,4,47.33,4.0,-25.840085,-39.653737,-21.33,-44.17
2014-01-01 00:00:00,36.80,6,49.19,6.0,-29.465971,-39.387976,-21.33,-44.17
2014-01-01 01:00:00,8.65,0,61.63,0.0,-9.269037,-60.928990,-25.89,-51.53
...,...,...,...,...,...,...,...,...
2024-11-17 22:00:00,224.35,6,32.92,6.0,23.012383,23.540532,11.94,23.99
2024-11-17 23:00:00,150.49,0,7.72,0.0,-3.802683,6.718482,15.46,28.58
2024-11-17 23:00:00,197.60,2,35.49,2.0,10.731107,33.828737,15.46,28.58


In [10]:
Flinten7.drop_duplicates(subset=["u_depth_averaged","v_depth_averaged"],inplace=True)
Flinten7

,Current direction,Measuring depth (m)_d,Current speed,Measuring depth (m)_s,u,v,u_depth_averaged,v_depth_averaged
Datum Tid (UTC),,,,,,,,
2014-01-01 00:00:00,4.27,0,57.52,0.0,-4.282743,-57.360339,-21.33,-44.17
2014-01-01 01:00:00,8.65,0,61.63,0.0,-9.269037,-60.928990,-25.89,-51.53
2014-01-01 02:00:00,16.32,0,73.05,0.0,-20.527176,-70.106616,-36.04,-64.43
2014-01-01 03:00:00,15.66,0,80.25,0.0,-21.661746,-77.271154,-40.98,-68.94
2014-01-01 04:00:00,19.07,0,87.21,0.0,-28.493520,-82.423925,-43.40,-75.13
...,...,...,...,...,...,...,...,...
2024-11-17 19:00:00,122.70,0,4.63,0.0,-3.896195,2.501313,12.78,26.51
2024-11-17 20:00:00,219.42,0,2.57,0.0,1.631951,1.985356,17.75,28.77
2024-11-17 21:00:00,179.13,0,1.54,0.0,-0.023383,1.539822,12.06,23.90


In [11]:
Flinten7[["u_depth_averaged","v_depth_averaged"]].rename(columns={"u_depth_averaged":"u","v_depth_averaged":"v"}).to_csv("../observations/Flinten7_u_v.csv", date_format='%Y-%m-%dT%H:%M:%SZ')

## Drogden and Helsingborg stations

In [12]:

dirs = ["smhi-opendata_2_3_35110_20241118_065837.csv","smhi-opendata_2_3_33101_20241118_065414.csv"]
stations = ["Helsingborg", "Drogden"]

for i,d in enumerate(dirs):
    data = pd.read_csv(f"raw_data/{d}",index_col=0,sep=";", parse_dates=True)
    data.rename(columns = swedish_to_english,inplace=True)

    # Filter data after time steps and quality flags.
    data = data.loc[(data.index >= "2014-01-01") & (data.index<"2024-01-01") & (data["Quality_d"] == "G") & (data["Quality_s"] == "G") ].drop(columns=["Quality_d","Quality_s","Unnamed: 7","Tidsutsnitt"])
    
    # Convert Current speed and directions to u and v components and calculate depth averaged velocities
    data = calculate_depth_averaged_velocities(data)
    data.drop_duplicates(subset=["u_depth_averaged","v_depth_averaged"],inplace=True)
    data[["u_depth_averaged","v_depth_averaged"]].rename(columns={"u_depth_averaged":"u","v_depth_averaged":"v"}).to_csv(f"../observations/{stations[i]}_u_v.csv", date_format='%Y-%m-%dT%H:%M:%SZ')
